In [1]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.

Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.

W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.

Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.

Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.

Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

In [28]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import re
import string
import networkx as nx
from collections import Counter

from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from typing import List
from functools import reduce
import operator
from gensim.models import Word2Vec
from IPython.core.display import display, HTML

In [4]:
# STOP_WORDS = set(stopwords.words('english'))
from spacy.lang.pl.stop_words import STOP_WORDS

In [5]:
vector = List[float]

def dot(v: vector, w: vector):
    return sum([vi * wi for vi, wi in zip(v,w)])

def cos_sim(v: vector, w: vector):
    return dot(v, w) / (dot(v,v) * dot(w,w)) ** .5


In [6]:
class Preprocessing(object):
    def __init__(self, text):
        self.text = text
        self.oryg = text

    def lower(self):
        self.text = self.text.lower() 
        return self.text
    
    def remove_punctuation(self):
        self.text = self.text.translate(self.text.maketrans('', '', string.punctuation.replace('.', '')))
        return self.text 
    
    def remove_stop_words(self):
        self.text = ' '.join([word for word in self.text.split() if word not in STOP_WORDS])
        return self.text
    
    def remove_digits(self):
        self.text = re.sub(r'[\d+]', '', self.text)
        return self.text
    
    def sentence_tokenize(self):
        self.text = sent_tokenize(self.text)
        self.text = [sent.replace('.','') for sent in self.text]
        return self.text
    
    def basic_pipeline(self):
        self.lower()
        self.remove_digits()
        self.remove_punctuation()
        self.remove_stop_words()
        self.sentence_tokenize()
        return self.text

    def __call__(self):
        return self.text

In [7]:
def prepare_text(text):
    cleaned_text = Preprocessing(text)
    sentences = cleaned_text.basic_pipeline()
    raw_sentences = Preprocessing(text).sentence_tokenize()
    return sentences, raw_sentences

In [8]:
sentences, raw_sentences = prepare_text(text)

In [9]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

In [25]:
all_words = [elem.split() for elem in sentences]
all_words_flattened = reduce(operator.concat, all_words)

In [41]:
def make_representation(tokens):
    def return_tokens(tokens):
        return [int(token in tokens) for token in all_words_flattened]
    
    X = list(map(return_tokens, tokens))

    return X

In [54]:
sim_matrix = np.zeros((len(sentences), len(sentences)))
for sent1 in range(len(sentences)):
    for sent2 in range(len(sentences)):
        if sent1 == sent2:
            continue         
        a = make_representation(sentences)[sent1]
        b = make_representation(sentences)[sent2]
        # print(cos_sim(a,b))
        sim_matrix[sent1][sent2] = cos_sim(a,b)

In [56]:
scores = nx.pagerank(nx.from_numpy_array(sim_matrix))
scores

{0: 0.04500595658518015,
 1: 0.004643962848297214,
 2: 0.007946805996030312,
 3: 0.008840043523319834,
 4: 0.02452590758266116,
 5: 0.024536906490381018,
 6: 0.04886718093620607,
 7: 0.021708080681938136,
 8: 0.04589805891417531,
 9: 0.04616203065140028,
 10: 0.01910958025757991,
 11: 0.04644167720935029,
 12: 0.0147064177114482,
 13: 0.03778582219689228,
 14: 0.017324279752864956,
 15: 0.012537223195066987,
 16: 0.04173633906397088,
 17: 0.04669483152551883,
 18: 0.004643962848297214,
 19: 0.054193780600936076,
 20: 0.01481968186964313,
 21: 0.05454866674084134,
 22: 0.04595526206564974,
 23: 0.009211582850948705,
 24: 0.04247855088417967,
 25: 0.013068699702118382,
 26: 0.04452716817492944,
 27: 0.04993150859523551,
 28: 0.020327696269464297,
 29: 0.028813846818924238,
 30: 0.011788223070272153,
 31: 0.030688248416153783,
 32: 0.045830059282719375,
 33: 0.014701956687405461}

In [59]:
summary = sorted(((scores[i], s) for i,s in enumerate(raw_sentences)), reverse = True)

In [74]:
summarized_sents = [sent[1] for sent in summary[:10]]
summarized_sents

['Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice)',
 'Jednak Gabriel nie jest usatysfakcjonowany',
 'Gabriel wraca do Vandroiya, który czekał na niego',
 'Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii',
 'Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower)',
 'Niosą ze sobą także księgę odkrytą przez Gabriela',
 'Vandroiy używa go by przenieść Gabriela do równoległego świata',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy',
 'Anna ucieka by ponownie złączyć się z Gabrielem']

In [75]:
def display_highlights(raw_sentences, summarized_sents):
    final = []
    for sent in raw_sentences:
        if sent not in summarized_sents:
            final += sent
        else:
            final += f'<span style="background-color:rgba(255,215,0,0.3);"> {sent} </span>'
    display(HTML(''.join([elem for elem in final])))

In [76]:
display_highlights(raw_sentences, summarized_sents)